<a href="https://colab.research.google.com/github/vrspi/Trade_Management/blob/main/Trade_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import yfinance as yf

data = pd.read_csv("USDJPY-2000-2020-15m.csv")
data=data[data.HIGH!=data.LOW]
data = data.reset_index()

data.head()


,index,DATE_TIME,HIGH,LOW,OPEN,CLOSE
0,0,2000.01.03 00:00:00,102.15,102.12,102.14,102.14
1,1,2000.01.03 00:15:00,102.26,102.13,102.15,102.26
2,2,2000.01.03 00:30:00,102.17,102.14,102.15,102.15
3,3,2000.01.03 00:45:00,102.20,102.12,102.16,102.20
4,4,2000.01.03 01:00:00,102.22,102.12,102.20,102.12


In [6]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=397c710c2603ef3dc055f634be7fa4c1d0e71167de39dda4604505d7b7859368
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [25]:
data.describe()

,index,HIGH,LOW,OPEN,CLOSE
count,500025.000000,500025.000000,500025.000000,500025.000000,500025.000000
mean,250169.503945,106.680786,106.580884,106.630679,106.630622
std,144395.187371,13.154707,13.140003,13.146951,13.146960
min,0.000000,75.696000,75.570000,75.642000,75.643000
25%,125131.000000,100.444000,100.336000,100.390000,100.391000
50%,250182.000000,109.078000,108.977000,109.030000,109.030000
75%,375226.000000,116.830000,116.700000,116.760000,116.760000
max,500233.000000,135.190000,134.940000,134.990000,134.980000


In [26]:
import pandas_ta as ta
data['EMA']=ta.sma(data.CLOSE, length=200)#sma ema
data['RSI']=ta.rsi(data.CLOSE, length=2)
#data.ta.indicators()
#help(ta.bbands)
my_bbands = ta.bbands(data.CLOSE, length=20, std=2.5)
my_bbands[0:50]
data=data.join(my_bbands)
data.dropna(inplace=True)
data.reset_index(inplace=True)
#data.set_index('Gmt time', inplace=True)
data[420:425]

,level_0,index,DATE_TIME,HIGH,LOW,OPEN,CLOSE,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5
420,619,621,2000.01.11 12:00:00,105.97,105.76,105.82,105.92,105.15755,98.133713,105.295749,105.6290,105.962251,0.630984,0.936608
421,620,622,2000.01.11 12:15:00,105.92,105.76,105.92,105.84,105.15945,52.481340,105.338745,105.6530,105.967255,0.594881,0.797529
422,621,623,2000.01.11 12:30:00,105.92,105.79,105.85,105.81,105.16175,38.906643,105.358265,105.6690,105.979735,0.588129,0.726881
423,622,624,2000.01.11 12:45:00,105.83,105.72,105.80,105.74,105.16490,17.628197,105.400561,105.6840,105.967439,0.536389,0.598787
424,623,625,2000.01.11 13:00:00,105.96,105.72,105.76,105.89,105.16940,75.366558,105.463218,105.7095,105.955782,0.465959,0.866450


In [27]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.HIGH[i]>=df.EMA[i]:
                dnt=0
            if df.LOW[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(data, 8)

In [28]:
def addorderslimit(df, percent):
    ordersignal=[0]*len(df)
    for i in range(1, len(df)): #EMASignal of previous candle!!! modified!!!
        if df.EMASignal[i]==2 and df.CLOSE[i]<=df['BBL_20_2.5'][i]:# and df.RSI[i]<=100: #Added RSI condition to avoid direct CLOSE condition
            ordersignal[i]=df.CLOSE[i]-df.CLOSE[i]*percent
        elif df.EMASignal[i]==1 and df.CLOSE[i]>=df['BBU_20_2.5'][i]:# and df.RSI[i]>=0:
            ordersignal[i]=df.CLOSE[i]+df.CLOSE[i]*percent
    df['ordersignal']=ordersignal

addorderslimit(data, 0.001)

In [29]:
data[data.ordersignal!=0]

,level_0,index,DATE_TIME,HIGH,LOW,OPEN,CLOSE,EMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,EMASignal,ordersignal
223,422,423,2000.01.07 10:00:00,105.370,105.090,105.310,105.110,104.523800,9.731688,105.138048,105.33300,105.527952,0.370163,-0.071936,2,105.004890
224,423,424,2000.01.07 10:15:00,105.150,104.870,105.130,104.990,104.533400,5.635727,105.050102,105.31900,105.587898,0.510636,-0.111756,2,104.885010
716,915,917,2000.01.14 14:15:00,105.850,105.660,105.670,105.830,105.891750,98.404547,105.509566,105.65200,105.794434,0.269629,1.124850,1,105.935830
856,1055,1058,2000.01.18 01:30:00,105.080,104.910,104.920,105.060,105.468300,96.829468,104.753854,104.88600,105.018146,0.251980,1.158364,1,105.165060
884,1083,1086,2000.01.18 08:30:00,105.120,104.930,104.950,105.070,105.297500,90.799030,104.867311,104.95650,105.045689,0.169954,1.136289,1,105.175070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499381,499580,499789,2020.03.12 14:30:00,104.208,103.818,103.849,104.137,104.475125,90.869561,103.444766,103.74895,104.053134,0.586384,1.137854,1,104.241137
499411,499610,499819,2020.03.12 22:00:00,105.369,104.727,105.328,104.737,104.596450,3.130376,104.772489,105.32900,105.885511,1.056711,-0.031885,2,104.632263
499511,499710,499919,2020.03.16 00:00:00,106.900,106.454,106.846,106.598,105.250840,1.477568,106.920893,107.89095,108.861007,1.798217,-0.166430,2,106.491402
499512,499711,499920,2020.03.16 00:15:00,106.598,105.934,106.598,105.987,105.258090,0.809716,106.429305,107.82410,109.218895,2.587167,-0.158556,2,105.881013


# Visualization

In [30]:
import numpy as np
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
data['pointposbreak'] = data.apply(lambda row: pointposbreak(row), axis=1)

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = data[1000:1250].copy()
#dfpl=dfpl.drop(columns=['level_0'])#!!!!!!!!!!
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['OPEN'],
                high=dfpl['HIGH'],
                low=dfpl['LOW'],   # Corrected here
                close=dfpl['CLOSE']),  # And here
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl['BBL_20_2.5'], line=dict(color='blue', width=1), name="BBL_20_2.5"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_20_2.5'], line=dict(color='blue', width=1), name="BBU_20_2.5")])


fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [32]:
dfpl = data[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [33]:
!pip install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173802 sha256=b8df601ac87fca07f0ca483a11efa808890a38b5cb931c3cf54fc2deddf6f9ef
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [35]:
dfpl.rename(columns={
    'OPEN': 'Open',
    'HIGH': 'High',
    'LOW': 'Low',
    'CLOSE': 'Close',
    # Include 'VOLUME': 'Volume' if you have a volume column
}, inplace=True)

In [40]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.05
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()

        for j in range(0, len(self.orders)):
            if self.data.index[-1]-self.ordertime[0]>15:#Period(15MIN) max to fulfill the order!!!
                self.orders[0].cancel()
                self.ordertime.pop(0)

        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
                #print(self.data.index[-1], self.trades[-1].entry_time)

            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()

        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            #Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            #Add new replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=500, margin=1/100, commission=.000)
stat = bt.run()
stat

<ipython-input-40-5769fa355ed4>:47: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                  499825.0
Duration                             499825.0
Exposure Time [%]                    0.711448
Equity Final [$]                   957.048223
Equity Peak [$]                    982.842933
Return [%]                          91.409645
Buy & Hold Return [%]                6.239557
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.452332
Avg. Drawdown [%]                   -1.250496
Max. Drawdown Duration                91768.0
Avg. Drawdown Duration            4281.194444
# Trades                                651.0
Win Rate [%]                        68.663594
Best Trade [%]                       0.620262
Worst Trade [%]                     -2.450532
Avg. Trade [%]                    

In [42]:
bt.plot(show_legend=False)

In [52]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,22,224,24154,105.130,113.51880,184.55360,0.079795,224,24154,23930
1,23,224,29540,105.130,121.92760,386.34480,0.159779,224,29540,29316
2,41,29583,49111,121.990,131.71680,398.79880,0.079734,29583,49111,19528
3,43,29583,61118,121.990,121.99000,0.00000,0.000000,29583,61118,31535
4,-57,61179,121155,122.490,102.96720,1112.79960,0.159383,61179,121155,59976
...,...,...,...,...,...,...,...,...,...,...
106,1,269899,269904,84.027,84.02700,0.00000,0.000000,269899,269904,5
107,1,269906,269932,83.977,83.97700,0.00000,0.000000,269906,269932,26
108,-1,270017,270018,83.776,83.77600,-0.00000,-0.000000,270017,270018,1
109,-1,270180,322048,82.636,89.24472,-6.60872,-0.079974,270180,322048,51868


In [47]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 0.05 #1000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2.
        perc = 0.02

        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            sl1 = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]+sldiff*TPSLRatio
            tp2 = self.data.Close[-1]+sldiff
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            sl1 = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]-sldiff*TPSLRatio
            tp2 = self.data.Close[-1]-sldiff
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=500, margin=1/100, commission=0.0000)
stat = bt.run()
stat

<ipython-input-47-93a372588d83>:31: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                  499825.0
Duration                             499825.0
Exposure Time [%]                   94.938038
Equity Final [$]                     50.75568
Equity Peak [$]                     3079.4006
Return [%]                         -89.848864
Buy & Hold Return [%]                6.239557
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -98.991786
Avg. Drawdown [%]                   -3.496298
Max. Drawdown Duration               338660.0
Avg. Drawdown Duration            1987.155378
# Trades                                424.0
Win Rate [%]                        41.273585
Best Trade [%]                        4.12587
Worst Trade [%]                     -2.112313
Avg. Trade [%]                    

In [50]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 0.05 #1000
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 2
        perc = 0.08

        if len(self.trades)==1:
            self.trades[-1].sl = self.trades[-1].entry_price

        if self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==2:
            sl1 = self.data.Close[-1]-self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]+sldiff*TPSLRatio
            tp2 = self.data.Close[-1]+sldiff
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1, tp=tp2, size=self.mysize)

        elif self.signal!=0 and len(self.trades)==0 and self.data.EMASignal==1:
            sl1 = self.data.Close[-1]+self.data.Close[-1]*perc
            sldiff = abs(sl1-self.data.Close[-1])
            tp1 = self.data.Close[-1]-sldiff*TPSLRatio
            tp2 = self.data.Close[-1]-sldiff
            self.sell(sl=sl1, tp=tp1, size=self.mysize)
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=500, margin=1/100, commission=0.0000)
stat = bt.run()
stat

<ipython-input-50-25c77e65efb9>:34: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                  499825.0
Duration                             499825.0
Exposure Time [%]                   61.032439
Equity Final [$]                      4.15736
Equity Peak [$]                      4257.332
Return [%]                         -99.168528
Buy & Hold Return [%]                6.239557
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.902348
Avg. Drawdown [%]                   -2.389619
Max. Drawdown Duration               374805.0
Avg. Drawdown Duration            1197.074341
# Trades                                111.0
Win Rate [%]                         7.207207
Best Trade [%]                      16.007649
Worst Trade [%]                     -8.011536
Avg. Trade [%]                    